# Init

In [1]:
using LinearAlgebra
using NearestNeighbors
using StaticArrays

# Utility Functions

In [2]:
########## RECTANGULAR GRIDS #######################################################################
"""
Get all possible combos of `tickLists` elements
"""
function regular_grid_helper( tickLists, i = 1, curVec = nothing, allPts = nothing )
    mDim = size( tickLists, 1 )
    
    # If this is the root call, then create the return matrix
    if allPts == nothing
        nPts = 1
        for tcs in tickLists
            nPts *= size( tcs, 1 )
        end
        allPts = zeros(mDim, nPts)
    end
    
    # If we are beginning a new vector
    if curVec == nothing
        curVec = []
    end
    
    # Base Case: Last Tick Dimension
    if mDim  == 1
        for tick in tickLists[1]
            nuVec       =  vcat(curVec, tick) 
            allPts[:,i] =  nuVec
            i += 1
        end
        curVec = nothing
    # Recursive Case
    else
        for tick in tickLists[1]
            _, i = regular_grid_helper( tickLists[2:end], i , vcat(curVec,tick), allPts )
        end
    end
    
    return allPts, i
end


"""
Return per-dim ticks of a regular grid of points across `rangeMatx` and `spacingArr` per-dim 
"""
function get_nD_ticks( rangeMatx, spacingArr )
    rtnTicks = []
    mDim     = size( rangeMatx, 1 )
    
    # 1. Generate tick marks in each dimension
    for j in 1:mDim 
        dimTicks = []
        bgn      = rangeMatx[j,1]
        stp      = rangeMatx[j,2]
        k        = 1
        append!( dimTicks, bgn )
        while true
            t_k = bgn + (k * spacingArr[j])
            if t_k <= stp
                append!( dimTicks, t_k )
            else
                break
            end
            k += 1
        end
        append!( rtnTicks, [dimTicks] )
    end
    return rtnTicks
end


"""
Return a regular grid of points across `rangeMatx` and per-dim space according to `spacingArr`
"""
function regular_grid_pts_nD( rangeMatx, spacingArr )
    tickLists = get_nD_ticks( rangeMatx, spacingArr )
    arr, _ = regular_grid_helper( tickLists )
    return arr
end

regular_grid_pts_nD

# kD Kernel

In [3]:
_DIM_X    = 2
_DIM_A    = 2
_X_DOMAIN = [-2.0 +2.0; -2.0 +2.0]
_A_DOMAIN = [-0.5 +0.5; -0.5 +0.5]
_Q_DOMAIN = [_X_DOMAIN; _A_DOMAIN]
_LEAFLEN  = 10

anchors = regular_grid_pts_nD( _X_DOMAIN, [0.25,0.25] );

nPts = size( anchors )[2]; # - Number of anchors
mDim = size( anchors )[1]; # - Dimensionality of anchors 
pnts = copy( anchors ); # ---- Interpolation anchors
vals = zeros(Float64, nPts); # Values at anchors
        
println( size( anchors ) )

(2, 289)


In [4]:
Q_kdTree = KDTree( pnts            ; leafsize = _LEAFLEN, reorder = false ); # Vals must remain assoc w pnts!

# println( pnts[1:_DIM_X,11] ) # Check X slice order
# println( pnts[1:_DIM_X,23] )
# println( pnts[1:_DIM_X,35] )

# X_kdTree = KDTree( pnts[1:_DIM_X,:]; leafsize = _LEAFLEN, reorder = false ); # Prohibit slice reorder

# println( pnts[1:_DIM_X,11] ) # Check X slice consistent 
# println( pnts[1:_DIM_X,23] )
# println( pnts[1:_DIM_X,35] )

In [5]:
"""
Return the anchors and values associated with the `idxList`
"""
function fetch_by_indices( pnts, vals, idxList )
    N = size( idxList, 1 ) # -------- How many queries are there?
    p = zeros( size( pnts, 1 ), N ) # Allocate points
    v = zeros( N ) # ---------------- Allocate values
    for i = 1:N
        j = idxList[i]
        p[:,i] = pnts[:,j]
        v[i]   = vals[j]
    end
    return p, v
end

"""
Compute a weighted average value from `vals`, `wgts`
NOTE: This function assumes that the scalar elements of `wgts` sum to 1.0
"""
function val_weighted_avg( vals, wgts )
    nPts = size( vals, 1 )
    r    = 0.0
    for j in 1:nPts
        v = vals[j]
        w = wgts[j]
        r += (w * v)
    end
    return r
end

val_weighted_avg

In [6]:
##### k Nearest Neighbors #####

"""
Return the indices and weights of the `k` nearest neighbors
NOTE: This function assumes that you have already built a kD-Tree and cached it in the `QStore`
"""
function K_kNN( kdTree, q; k = 6 )
    factor      = 1.0/k
    idxs, dists = knn( kdTree, q, k )
    return idxs, repeat( [factor], k )
end


"""
Use k Nearest Neighbors of `q` to retrive value from a `QStore`
"""
function query_value_w_knn( kdTree, pnts, vals, q; k = 6 )
    idxs, wgts = K_kNN( kdTree, q; k = k )
    p_  , v_   = fetch_by_indices( pnts, vals, idxs )
    return val_weighted_avg( v_, wgts )
end

query_value_w_knn

In [7]:
########## TESTING FUNCTIONS #######################################################################

"""
Hyperplane with a uniform positive slope along all axes
"""
function get_nD_example_plane_eq( dims, slope )
    factor = slope / dims
    Y = repeat( [factor], dims )
    return function( X )
        dot( X, Y )
    end
end


"""
Cone of `height` that slopes downward from origin 
"""
function get_nD_example_cone_eq( height, slope )
    return function( X )
        height - norm( X ) * slope
    end
end


"""
Hill of `height` that slopes downward from origin 
"""
function get_nD_example_hill_eq( height, scale )
    return function( X )
        height * exp( -norm(X)/scale )
    end
end

get_nD_example_hill_eq

In [8]:
"""
Use `truthFunc` to load the correct values for each anchor in `qSto`
NOTE: This function assumes that `truthFunc` has the same `mDim` as `truthFunc`
"""
function assoc_pnts_with_true_values( pnts, vals, truthFunc )
    # For every column of stored points, compute the truth value
    nPts = size( pnts, 2 )
    for j in 1:nPts
        vals[j] = truthFunc( pnts[:,j] )
    end
end

assoc_pnts_with_true_values

In [9]:
########## UNIFORM SAMPLING ########################################################################

"""
Sample from a uniform distribution [ rMin , rMax )
"""
function sample_uniform_real( rMin , rMax )
    span = abs( rMax - rMin )
    return span*rand() + rMin
end


"""
Sample uniformly from a hypercuboid `bbox`
"""
function sample_bbox_uniform( bbox )
    mDim   = size( bbox, 1 )
    rtnArr = zeros( mDim )
    # print( mDim, ' ', rtnArr )
    # display( bbox )
    for i = 1:mDim
        rMin = bbox[i,1]
        # println( "rMin:", rMin )
        rMax = bbox[i,2]
        # println( "rMax:", rMax )
        rtnArr[i] = sample_uniform_real( rMin , rMax )
    end
    return rtnArr
end


"""
Draw `N` samples uniformly from a hypercuboid `bbox`
"""
function sample_bbox_N( bbox, N )
    mDim   = size( bbox, 1 )
    rtnArr = zeros( mDim, N )
    for j = 1:N
        rtnArr[:,j] = sample_bbox_uniform( bbox )
    end
    return rtnArr
end

sample_bbox_N

In [10]:
truthFunc = get_nD_example_plane_eq( _DIM_X, 1 )
assoc_pnts_with_true_values( pnts, vals, truthFunc )
display( query_value_w_knn( Q_kdTree, pnts, vals, [2; 2;]; k = 6 ) )

1.8333333333333333

In [11]:
bbox = _X_DOMAIN
N    = 50
smpl = sample_bbox_N( bbox, N )
err  = 0.0

for j = 1:N
    smp = smpl[:,j]
    est = query_value_w_knn( Q_kdTree, pnts, vals, smp; k = 3 )
    tru = truthFunc( smp )
    err += abs( est - tru )
end

println( "Average error for knn kernel on plane: ", err/N )

Average error for knn kernel on plane: 0.023768380965618584


## Radius Kernel

In [12]:
##### Fixed Radius #####

"""
Return the indices and weights of the neighbors within `rad` of `d`
NOTE: This function assumes that you have already built a Ball Tree and cached it in the `QStore`
"""
function K_fixed_rad( blTree, q; rad = 1.0 )
    idxs     = inrange( blTree, q, rad )
    n        = size( idxs, 1 )
    factor   = 1.0/n
    return idxs, repeat( [factor], n )
end


"""
Use a ball within `radius` of `q` to retrive value from a `QStore`
"""
function query_store_w_radius( blTree, pnts, vals, q; radius = 1.0 )
    idxs, wgts = K_fixed_rad( blTree, q; rad = radius )
    p_  , v_   = fetch_by_indices( pnts, vals, idxs )
    return val_weighted_avg( v_, wgts )
end

query_store_w_radius

In [13]:
Q_blTree = BallTree( pnts             ); # Vals must remain assoc w pnts!
# X_blTree = BallTree( pnts[1:_DIM_X,:] ); # Prohibit slice reorder

In [14]:
bbox = _X_DOMAIN
N    = 50
smpl = sample_bbox_N( bbox, N )
err  = 0.0

for j = 1:N
    smp = smpl[:,j]
    est = query_store_w_radius( Q_blTree, pnts, vals, smp; radius = 1.0 )
    tru = truthFunc( smp )
    err += abs( est - tru )
end

println( "Average error for knn kernel on plane: ", err/N )

Average error for knn kernel on plane: 0.06471225586396001


## Gaussian Kernel

In [15]:
##### (Truncated) Gaussian Kernel #####


"""
Used by the Gaussian Kernel
"""
function gaussian( s )
    exp( -s^2 / 2.0 )
end


"""
Return the indices and weights of the neighbors chosen by the Truncated Gaussian Function
"""
# FIXME: REWRITE SO THAT IT ONLY SEARCHES WITHIN A RADIUS!!!

function K_trunc_Gauss( pnts, q; h = 1.0, lim = 1.0 )
    idxs = [] # ------------ Index associated with each weight
    wght = [] # ------------ The weight of each index
    totl = 0.0 # ----------- Total of all distance calcs
    nPts = size( pnts, 2 ) # Total number of points/anchors
    
    # For each anchor point in our collection
    for j in 1:nPts
        pnt = pnts[:,j] # Fetch point
        s_j = norm(q-pnt)/h #- Compute the distance from the query
        # If the distance is equal to or less than the distance limit, then consider an applicable point
        if s_j <= lim
            g_j = gaussian( s_j ) # Compute Gaussian distance
            append!( idxs, j   ) #- Store index
            append!( wght, g_j ) #- Store distance
            totl += g_j # --------- Sum distance
        end
    end
    wght ./= totl # Normalize the weights (MUST sum to 1.0)
    return idxs, wght # Return applicable points and their weights
end


"""
Use a Gaussian kernel centered on `q` to retrive value from a `QStore`
"""
function query_value_w_gauss( pnts, vals, q; h = 1.0, lim = 1.0 )
    idxs, wgts = K_trunc_Gauss( pnts, q; h = h, lim = lim )
    p_  , v_   = fetch_by_indices( pnts, vals, idxs )
    return val_weighted_avg( v_, wgts )
end

query_value_w_gauss

In [16]:
err  = 0.0

for j = 1:N
    smp = smpl[:,j]
    est = query_value_w_gauss( pnts, vals, smp )
    tru = truthFunc( smp )
    err += abs( est - tru )
end

println( "Average error for radius kernel on hill: ", err/N )

Average error for radius kernel on hill: 0.05710057574777812
